In [53]:
import os
import boto3
from tqdm import tqdm
from glob import glob
from pydub import AudioSegment
from botocore.exceptions import ClientError

In [54]:
def print_buckets(s3_client):
    response = s3_client.list_buckets()

    print('Existing buckets:')
    for bucket in response['Buckets']:
        print(f'  {bucket["Name"]}')

def upload_file(s3_client, file_name, bucket, object_name = None):
    if object_name is None:
        object_name = os.path.basename(file_name)   
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)

def convert_mp3(source, destination):
    sound = AudioSegment.from_mp3(source)
    sound.export(destination, format = 'wav')        
        
def print_files_in_buckets(bucket_name):
    s3 = boto3.resource('s3')
    my_bucket = s3.Bucket(bucket_name)

    for object_summary in my_bucket.objects.filter():
        print(f'-- {object_summary.key}')
    print()

In [55]:
s3_client = boto3.client('s3')

files = glob('../Data/cv-corpus/en/clips/*.mp3')
for file in tqdm(files):
    convert_mp3(file, file.replace('.mp3', '.wav'))
    
files = glob('../Data/cv-corpus/en/clips/*.wav')
for file in tqdm(files):
    upload_file(s3_client = s3_client, 
        file_name = file, 
        bucket = 'audio-data-wake-word',
        object_name = 'cv_corpus_wav/' + file.split('/')[-2] + '_' + file.split('/')[-1]
    )

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26883/26883 [2:55:36<00:00,  2.55it/s]
